<a href="https://colab.research.google.com/github/ItshMoh/PDF_Summarizer/blob/main/Pdf_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer.six
!pip install transformers

In [ ]:
#Run this code to give access the pdf which you want to summarize.
#Either you can upload the pdf to the current session.
from google.colab import drive
drive.mount('/content/drive')


In [2]:
from pdfminer.high_level import extract_pages

In [4]:
from transformers import pipeline

In [ ]:
# Here you can do the summarization with other models. Take a look at the huggingmodels.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO

# Open the PDF file
pdf_file = open('/path/to/your/pdf', 'rb')

# Create a PDF resource manager and set parameters
resource_manager = PDFResourceManager()
output_stream = BytesIO()
laparams = LAParams()

# Create a PDF page interpreter
pdf_page_interpreter = PDFPageInterpreter(resource_manager, TextConverter(resource_manager, output_stream, laparams=laparams))

# Specify the page number you want to access (0-based index)
page_number = [i] # Replace i with the page number you need you can give two or more page numbers seperated with comma.

# Access the specified page's text
pdf_file.seek(0)  # Reset the file pointer to the beginning
pdf_pages = PDFPage.get_pages(pdf_file, caching=True, check_extractable=True)
for i, page in enumerate(pdf_pages):
  for j in page_number:

    if i+1 == j:
        pdf_page_interpreter.process_page(page)

# Get the page text
page_text = output_stream.getvalue()

# Close the PDF file and output stream
pdf_file.close()
output_stream.close()

# Print or use the page text as needed
extracted_text=page_text.decode('utf-8')


In [ ]:
#Run this cell if you just want to access the text of the page you specify'
#print(extracted_text)

In [8]:
import textwrap
words_size=700
batches= textwrap.wrap(extracted_text,width=words_size)

In [ ]:
#Run the below code if you want to see the batches of the pages.
# for i, batch in enumerate(batches):
#   print(f"Batch {i+1}\n {batch}\n")

In [ ]:
overall_summary=[]
import re
for i, batch in enumerate(batches):
  summary= summarizer(batch,max_length=75,min_length=25,do_sample=False)

  overall_summary.append(summary)

summaries= " " .join(map(str,overall_summary))
pattern = r'[{\"\':}]'
substring_to_remove="summary_text"
cleaned_summary= re.sub(pattern,'',summaries)
Required_summary= cleaned_summary.replace(substring_to_remove,"")
Required_summary